In [13]:
df = spark.read.option("multiline","true").json("Files/paises_json")
display(df)


StatementMeta(, f0ed1548-0332-430e-a865-f191239fd528, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 38516a14-f7f8-43d4-90ea-e74fea9e707a)

In [14]:
from pyspark.sql.functions import col, element_at

df_silver = df.select(
    col("name.common").alias("nome_pais"),
    col("cca2").alias("codigo_iso_2"),
    col("cca3").alias("codigo_iso_3"),
    col("region").alias("regiao"),
    col("subregion").alias("subregiao"),
    col("population").alias("populacao"),
    col("area").alias("area_km2"),
    element_at(col("capital"), 1).alias("capital"),
    col("flag"),
    col("independent").alias("independencia"),
    col("unMember").alias("membro_Onu"),
    col("landlocked").alias("sem_saida_para_o_mar"),
    col("startOfWeek").alias("dia_inicial_semana"),
    col("fifa").alias("codigo_fifa"),
    col("latlng").getItem(0).alias("latitude"),
    col("latlng").getItem(1).alias("longitude")
)

display(df_silver)


StatementMeta(, f0ed1548-0332-430e-a865-f191239fd528, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d4c22c19-99cc-4ec5-bdf3-d0383b4e3455)

In [15]:
from pyspark.sql.functions import col, trim, upper, initcap

df_tratado = df_silver \
    .withColumn("nome_pais", trim(initcap("nome_pais"))) \
    .withColumn("codigo_iso_2", trim(upper("codigo_iso_2"))) \
    .withColumn("regiao", trim(initcap("regiao"))) \
    .withColumn("subregiao", trim(initcap("subregiao"))) \
    .withColumn("capital", trim(initcap("capital"))) \
    .withColumn("dia_inicial_semana", trim(initcap("dia_inicial_semana")))

display(df_tratado)


StatementMeta(, f0ed1548-0332-430e-a865-f191239fd528, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fad2e2a2-0c12-4247-a8a3-d86e9dffe40a)

In [16]:
df_tratado = df_tratado.select(
    "nome_pais","codigo_iso_2","codigo_iso_3","regiao","subregiao",
    "populacao","area_km2","capital","flag","independencia",
    "membro_Onu","sem_saida_para_o_mar","dia_inicial_semana",
    "codigo_fifa","latitude","longitude"
)

display(df_tratado)


StatementMeta(, f0ed1548-0332-430e-a865-f191239fd528, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d5b5b154-49c7-4985-a5d6-55628e363d1a)

In [17]:
from pyspark.sql.functions import col

df_tratado = df_tratado.dropDuplicates(["codigo_iso_2", "codigo_iso_3"])
df_tratado = df_tratado.withColumn("populacao", col("populacao").cast("int"))

# Tratamento de nulos
df_tratado = df_tratado.fillna({
    'codigo_iso_2': 'Não informado', 
    'codigo_iso_3': 'Não informado',
    'nome_pais': 'Não informado',
    'capital': 'Não informado',
    'regiao': 'Não informado',
    'subregiao': 'Não informado',
    'dia_inicial_semana': 'Não informado',
    'populacao': 0,
    'area_km2': 0,
    'latitude': 0,
    'longitude': 0,
    'independencia': False,
    'membro_Onu': False,
    'sem_saida_para_o_mar': False
})

display(df_tratado)


StatementMeta(, f0ed1548-0332-430e-a865-f191239fd528, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3263c8cb-e373-4e53-948e-0edb2643c12a)

In [18]:
from pyspark.sql.functions import round, col

colunas_para_arredondar = ["area_km2", "latitude", "longitude"]

for nome_coluna in colunas_para_arredondar:
    df_tratado = df_tratado.withColumn(
        nome_coluna, 
        round(col(nome_coluna).cast("double"), 3)
    )

display(df_tratado)


StatementMeta(, f0ed1548-0332-430e-a865-f191239fd528, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b5e3591e-f0a5-4895-a0ea-ac094fed6e8f)

In [19]:
from pyspark.sql.functions import regexp_replace, col

df_tratado = df_tratado.withColumn("area_km2", col("area_km2").cast("string"))
df_tratado = df_tratado.withColumn("area_km2", regexp_replace("area_km2", "[Ee]", ""))
df_tratado = df_tratado.withColumn("area_km2", col("area_km2").cast("double"))

display(df_tratado)


StatementMeta(, f0ed1548-0332-430e-a865-f191239fd528, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a2c833fd-1236-4afa-9dc1-b0f8632639db)

In [20]:
# #Normalização dos números para km2
# df_tratado = df_tratado.withColumn(
#     "area_km2_corrigida",
#     when(col("area_km2") > 1000000, col("area_km2") / 1000000).otherwise(col("area_km2"))
# )
# df_tratado = df_tratado.withColumn(
#     "area_valida",
#     (col("area_km2_corrigida") > 0.1) & (col("area_km2_corrigida") < 20000000)
# )
# df_tratado.select("nome_pais", "area_km2", "area_km2_corrigida", "area_valida").show(45)


StatementMeta(, f0ed1548-0332-430e-a865-f191239fd528, 22, Finished, Available, Finished)

In [21]:
df_tratado.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("regiao") \
    .saveAsTable("tabela_paises")


StatementMeta(, f0ed1548-0332-430e-a865-f191239fd528, 23, Finished, Available, Finished)

In [22]:
%%sql
describe tabela_paises;

StatementMeta(, f0ed1548-0332-430e-a865-f191239fd528, 24, Finished, Available, Finished)

<Spark SQL result set with 19 rows and 3 fields>